## BASE DE DONNEES VECTORIELLE ET EMBEDING

## ETAPE 1 CHUNKING

In [24]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def load_and_split_pdf_simple(pdf_path: str):
    """
    Charge tout le PDF en un seul Document, puis le découpe en chunks
    avec chunk_size=1000 et chunk_overlap=100 (en caractères).
    """
    # 1. Extraire tout le texte du PDF
    reader = PdfReader(pdf_path)
    full_text = ""
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        full_text += text + f"\n\n[PAGE {i+1}]\n\n"  # Ajout de marqueur de page

    # 2. Créer un seul Document
    doc = Document(page_content=full_text.strip().replace('. .',''), metadata={"source": pdf_path})

    # 3. Découper avec RecursiveCharacterTextSplitter (par défaut : len = caractères)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['.'],
        keep_separator=True  # Garde les séparateurs pour éviter de couper au milieu d'une phrase
    )

    # 4. Splitter
    chunks = text_splitter.split_documents([doc])

    return chunks

# Utilisation
pdf_chunks = load_and_split_pdf_simple("embedding_course_by_koulou.pdf")




In [25]:
pdf_chunks

[Document(page_content='L’INTELLIGENCE ARTIFICIELLE\nDes fondements théoriques aux bases de données vectorielles\nAuteur : KOULOU crepin\nDépartement de Data Science\n8 janvier 2026\n\n[PAGE 1]\n\nRésumé\nCe rapport présente une analyse approfondie de l’état actuel de l’Intelligence Artiﬁcielle.\nNous explorerons d’abord les fondements mathématiques de l’apprentissage automatique\n(Machine Learning) et des réseaux de neurones profonds (Deep Learning). Une attention\nparticulière sera portée aux développements récents concernant les Grands Modèles de\nLangage (LLM) et l’infrastructure critique qui les soutient, notamment les bases de don-\nnées vectorielles (FAISS, ChromaDB, Qdrant) et les mécanismes de similarité. Enﬁn, nous\naborderons les déﬁs éthiques et les perspectives d’avenir.\n\n[PAGE 2]\n\nTable des matières\n1 Introduction 2\n1.1 Contexte Historique                . 2\n1.2 Déﬁnitions et Typologie               2\n2 Fondements Mathématiques du Machine Learning 4\n2.1 L’Apprent


<img src="faiss chroma.jpg" alt="Logo Python" width="1500" height="600">


<div style="display: flex; gap: 20px;">

  <!-- Colonne 1 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 1. Introduction Générale : Les Bases de Données Vectorielles

  **Qu'est-ce que c'est ?**  
  Une base de données vectorielle est un système optimisé pour stocker et interroger des *vecteurs* (embeddings).

  **Pourquoi en avons-nous besoin ?**  
  Les bases traditionnelles (SQL, NoSQL) gèrent bien les correspondances exactes, mais échouent sur la *similarité*.  

  Avec l’essor de l’IA, les données complexes (texte, images, audio) sont transformées en vecteurs.  
  ✨ Des concepts proches se retrouvent mathématiquement voisins dans un espace multidimensionnel.  

  👉 Fonction principale : **Vector Search**.  
  Exemple : un vecteur "chat" renvoie les images les plus proches d’un chat, même sans mot-clé.  

  C’est le cœur des applications modernes comme le **RAG (Retrieval-Augmented Generation)**, qui permet aux LLM d’accéder à vos documents.  

  </div>

  <!-- Colonne 2 -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 2. FAISS et ChromaDB : Solutions orientées "Développement Local"

  **FAISS (Facebook AI Similarity Search)**  
  - Origine : Meta (Facebook AI Research).  
  - Nature : Bibliothèque C++/Python, ultra performante.  
  - Forces :  
    - 🚀 Recherche sur milliards de vecteurs.  
    - ⚡ Optimisation GPU.  
  - Faiblesses :  
    - Bas niveau, pas de stockage persistant.  
    - Risque de perte si non sauvegardé.  

  **ChromaDB**  
  - Origine : Projet Open Source récent.  
  - Nature : Base vectorielle simple et rapide.  
  - Forces :  
    - 🎯 Installation facile (`pip install chromadb`).  
    - 🔄 Vectorisation intégrée.  
  - Faiblesses :  
    - Idéal pour petits/moyens projets.  
    - Moins robuste pour téraoctets en production.  

  </div>

</div>


## FAISS

In [26]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

def build_faiss_index(chunks):
    """
    Construit un index FAISS à partir de chunks déjà préparés.
    Utilise un modèle gratuit Hugging Face pour générer les embeddings.
    """
    # 1. Créer les embeddings avec Hugging Face
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"  # modèle open-source ou openaiembeddings
    )

    # 2. Construire l’index FAISS
    vectorstore = FAISS.from_documents(chunks, embeddings)

    return vectorstore



In [29]:
# Supposons que tu as déjà tes chunks :
# chunks = text_splitter.split_documents(documents)

faiss_index = build_faiss_index(pdf_chunks)

# Faire une requête
query = "Explique-moi la différence entre FAISS et ChromaDB"
results = faiss_index.similarity_search(query, k=3)

# Afficher les résultats
for i, res in enumerate(results):
    print(f"--- Résultat {i+1} ---")
    print(res.page_content[:500], "...")


--- Résultat 1 ---
.
8

[PAGE 10]

État de l’art Rapport IA 2024
4.3.1 Comparatif Technique
— FAISS (Meta): Bibliothèque bas niveau, optimisée pour la recherche par simila-
rité dense et le clustering. Utilise l’indexation IVF.
— ChromaDB : Solution open-source conviviale pour les développeurs, idéale pour
les applications LLM légères et le prototypage.
— Pinecone / Qdrant: Solutions gérées (SaaS) ou robustes (Rust) pour la produc-
tion à grande échelle.
4.3.2 Exemple d’implémentation (Python)
Voici comment on ini ...
--- Résultat 2 ---
. Dans ce contexte, les réseaux de neurones profonds et les
bases de données vectorielles jouent un rôle crucial pour traiter des informations non
structurées. Ce paragraphe sert de remplissage pour simuler le volume de texte du rap-
port ﬁnal, permettant d’apprécier la mise en page sans être distrait par du latin. Les
avancées technologiques récentes, notamment les architectures Transformers, ont ouvert
la voie aux modèles de langage massifs (LLM).
4.2

## CHROMA DB

In [30]:

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

def build_chroma_index(chunks, persist_directory="chroma_index"):
    """
    Construit un index Chroma à partir de chunks déjà préparés.
    Utilise un modèle gratuit Hugging Face pour générer les embeddings.
    Les données sont persistées dans un dossier local.
    """
    # 1. Créer les embeddings avec Hugging Face
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"  # modèle open-source
    )

    # 2. Construire l’index Chroma
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory  # dossier où sauvegarder l’index
    )

    # 3. Sauvegarder l’index
    vectorstore.persist()

    return vectorstore



In [31]:
# Supposons que tu as déjà tes chunks :
# chunks = text_splitter.split_documents(documents)

# Construire l’index
chroma_index = build_chroma_index(pdf_chunks)

# Faire une requête
query = "Explique-moi la différence entre FAISS et ChromaDB"
results = chroma_index.similarity_search(query, k=3)

# Afficher les résultats
for i, res in enumerate(results):
    print(f"--- Résultat {i+1} ---")
    print(res.page_content[:500], "...")


--- Résultat 1 ---
.
8

[PAGE 10]

État de l’art Rapport IA 2024
4.3.1 Comparatif Technique
— FAISS (Meta): Bibliothèque bas niveau, optimisée pour la recherche par simila-
rité dense et le clustering. Utilise l’indexation IVF.
— ChromaDB : Solution open-source conviviale pour les développeurs, idéale pour
les applications LLM légères et le prototypage.
— Pinecone / Qdrant: Solutions gérées (SaaS) ou robustes (Rust) pour la produc-
tion à grande échelle.
4.3.2 Exemple d’implémentation (Python)
Voici comment on ini ...
--- Résultat 2 ---
. Dans ce contexte, les réseaux de neurones profonds et les
bases de données vectorielles jouent un rôle crucial pour traiter des informations non
structurées. Ce paragraphe sert de remplissage pour simuler le volume de texte du rap-
port ﬁnal, permettant d’apprécier la mise en page sans être distrait par du latin. Les
avancées technologiques récentes, notamment les architectures Transformers, ont ouvert
la voie aux modèles de langage massifs (LLM).
4.2

# explication du fonctionnement de l algo de recherche et mesure de similarite

## <img src="cosine_similarity.jpg" alt="Logo Python" width="1500" height="600">


<div style="display: flex; gap: 20px;">

  <!-- Colonne 1 : Mathématique -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 1. La Mathématique : Mesurer la distance

  Dans une base vectorielle, chaque donnée (mot, phrase, image) est un point dans un espace multidimensionnel (souvent 768, 1536 ou plus).  
  Pour savoir si deux points sont "similaires", on mesure la **distance** ou l’**angle** entre eux.  

  ### 🔑 Les 3 métriques principales
  **A. Similarité Cosinus (Cosine Similarity)**  
  - Mesure l’angle entre deux vecteurs.  
  - 0° → similarité = 1 (max).  
  - 180° → similarité = -1.  
  - 90° → aucune relation.  
  - Usage : NLP et recherche de texte.  

  **B. Distance Euclidienne (L2)**  
  - Distance "à vol d’oiseau" entre deux points.  
  - Plus la valeur est petite, plus les vecteurs sont proches.  
  - Usage : reconnaissance d’images, magnitude importante.  

  **C. Produit Scalaire (Dot Product)**  
  - Multiplication des coordonnées des vecteurs.  
  - Usage : systèmes de recommandation, apprentissage hybride.  

  </div>

  <!-- Colonne 2 : Algorithmique -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 2. L'Algorithmique : Trouver vite (ANN)

  La méthode naïve (KNN) compare chaque document un par un → trop lent (O(N)).  
  Les bases vectorielles utilisent des **algorithmes ANN (Approximate Nearest Neighbors)** pour gagner en vitesse.  

  ### 🚀 Les 2 algorithmes stars
  **A. HNSW (Hierarchical Navigable Small World)**  
  - Graphe multi-couches : grands sauts (avion), moyens (train), petits (marche).  
  - Avantages : rapide et précis.  
  - Inconvénient : consomme beaucoup de RAM.  

  **B. IVF (Inverted File Index – FAISS)**  
  - Division de l’espace en régions (cellules de Voronoi).  
  - Recherche uniquement dans la région la plus proche.  
  - Avantages : réduit l’espace de recherche.  
  - Inconvénient : nécessite un entraînement initial (clustering).  

  ### 📊 En résumé
  | Concept   | Rôle                          | Exemple                        |
  |-----------|-------------------------------|--------------------------------|
  | Embedding | Transformer la donnée en chiffres | [0.1, -0.5, 0.8...]            |
  | Métrique  | Mesurer la distance           | Cosine vs Euclidean            |
  | ANN Algo  | Chercher vite                 | HNSW (graphe multi-niveaux)    |

  </div>

</div>


## ORIENTE CLOUD

Pinecone (SaaS, cloud géré) : https://www.pinecone.io/

Qdrant (Open Source et Cloud) : https://qdrant.tech/

<img src="vectora ligne.jpg" alt="Logo Python" width="1500" height="600">

<div style="display: flex; gap: 20px;">

  <!-- Colonne 1 : Pinecone -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 3. Pinecone : Le service Cloud géré (SaaS)

  **Nature**  
  Base de données vectorielle entièrement gérée dans le cloud.  
  👉 Vous n’installez rien sur vos serveurs.  

  **Forces**  
  - 🛠️ **Zéro maintenance** : infrastructure, scaling, sécurité et sauvegardes sont pris en charge.  
  - 🔒 **Fiabilité** : haute disponibilité et performances garanties (SLA).  
  - 🎯 Choix idéal pour les entreprises qui ne veulent pas gérer d’infrastructure.  

  **Faiblesses / Cas d’usage**  
  - 💰 **Coût élevé** si volume de données ou trafic important.  
  - 🌐 **Données externes** : vos données quittent votre infrastructure pour aller sur les serveurs Pinecone.  
  - 🔒 Logiciel propriétaire (fermé).  

  </div>

  <!-- Colonne 2 : Qdrant -->
  <div style="flex: 1; padding: 15px; border: 2px solid #e65100; border-radius: 10px; background-color: #fff0e0; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); color:#212121;">
  
  ## 3. Qdrant : Le serveur robuste et flexible (Open Source)

  **Nature**  
  Serveur de base de données vectorielle Open Source, écrit en **Rust** (langage rapide et sécurisé).  

  **Forces**  
  - 🔍 **Recherche Hybride (Filtrage)** : combine recherche vectorielle et filtres traditionnels.  
    Exemple : *« Trouve-moi des documents similaires, MAIS seulement s’ils datent de 2024 et sont de catégorie Finance »*.  
  - ⚡ **Performance et stabilité** grâce à Rust.  
  - 🐳 Hébergement possible via **Docker** (gratuit) ou version cloud gérée.  

  **Faiblesses / Cas d’usage**  
  - ✅ Excellent choix pour garder le contrôle des données (auto-hébergement).  
  - 🔧 Demande plus de configuration initiale que ChromaDB.  

  </div>

</div>


In [ ]:

# Installer les dépendances si besoin :
# pip install pinecone-client langchain sentence-transformers

import os
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone

# 1. Initialiser Pinecone
# Mets ta clé API et ton environnement (trouvés dans ton compte Pinecone)
pinecone_api_key = "YOUR_PINECONE_API_KEY"
pinecone_environment = "us-east1-gcp"  # ex: "us-east1-gcp"

pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)

# 2. Créer les embeddings avec Hugging Face (modèle gratuit)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 3. Créer l’index Pinecone (si non existant)
index_name = "pdf-index"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, dimension=384)  
    # 384 = dimension du modèle MiniLM-L6-v2

# 4. Construire le vectorstore Pinecone à partir de tes chunks
# Supposons que tu as déjà tes chunks (liste de Document LangChain)
# chunks = text_splitter.split_documents(documents)

vectorstore = Pinecone.from_documents(
    documents=pdf_chunks,
    embedding=embeddings,
    index_name=index_name
)

# 5. Faire une requête de similarité
query = "Explique-moi la différence entre FAISS et ChromaDB"
results = vectorstore.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"--- Résultat {i+1} ---")
    print(res.page_content[:300], "...")



## Ressource complementaire

https://www.youtube.com/watch?v=6rATNqmpvmQ

https://www.youtube.com/watch?v=jPYOrJHDXAQ